# <div style="font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:200%; text-align:center;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0">Santa 2023 - Iterative replacement of K successive moves</div>
#### <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:150%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >TABLE OF CONTENTS<br><div>
* [IMPORTS](#1)
* [LOAD DATA](#2)
* [FUNCTIONS](#3)
* [SOLVE](#4)

Code modified from: https://www.kaggle.com/code/zaburo/iterative-replacement-of-k-successive-moves

<a id="1"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" > IMPORTS<br><div> 

In [1]:
import argparse
import time

import numpy as np
import pandas as pd
from tqdm import tqdm
import zipfile
import sqlite3

<a id="2"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >LOAD DATA<br><div> 

In [2]:
with zipfile.ZipFile('../../../res/data/santa-2023.zip', 'r') as z:
    
    with z.open('puzzle_info.csv') as f:
        puzzle_info = pd.read_csv(f, index_col = 'puzzle_type')        
                
    with z.open('puzzles.csv') as f:
        puzzles = pd.read_csv(f, index_col = 'id')
    
    with z.open('sample_submission.csv') as f:
        submission = pd.read_csv(f)

In [3]:
allowed_moves = {}

for idx, row in puzzle_info.iterrows():
    allowed_moves[idx] = eval(row['allowed_moves'])

<a id="3"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >FUNCTIONS<br><div> 

In [4]:
class ExceedMaxSizeError(RuntimeError):
    pass


def get_shortest_path(
    moves: dict[str, tuple[int, ...]], K: int, max_size: int | None
) -> dict[tuple[int, ...], list[str]]:
    n = len(next(iter(moves.values())))

    state = tuple(range(n))
    cur_states = [state]

    shortest_path: dict[tuple[int, ...], list[str]] = {}
    shortest_path[state] = []

    for _ in range(100 if K is None else K):
        next_states = []
        for state in cur_states:
            for move_name, perm in moves.items():
                next_state = tuple(state[i] for i in perm)
                if next_state in shortest_path:
                    continue
                shortest_path[next_state] = shortest_path[state] + [move_name]

                if max_size is not None and len(shortest_path) > max_size:
                    raise ExceedMaxSizeError

                next_states.append(next_state)
        cur_states = next_states

    return shortest_path


def get_moves(puzzle_type: str) -> dict[str, tuple[int, ...]]:
    moves = eval(puzzle_info.loc[puzzle_type, "allowed_moves"])
    for key in list(moves.keys()):
        moves["-" + key] = list(np.argsort(moves[key]))
    return moves



<a id="4"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >SOLVE<br><div> 

In [5]:
database_file = '../solutions.db'
solution_method = 'iterative replacement of K successive moves'


conn = sqlite3.connect(database_file)
cursor = conn.cursor()

for game_id in range(381, 398):# range(187, 281):
    # game_id = 1
    puzzle = puzzles.loc[game_id]
    time_limit = 72 * 60 * 60
    
    select_query = "SELECT moves, solution_method FROM solutions WHERE id = ?"
            
    # Execute the query
    cursor.execute(select_query, (game_id,))
    response = cursor.fetchone()
    
    sample_moves = response[0].split(".")
    best_solution_method = response[1]
    
    # sample_submission = pd.read_csv("data/sample_submission.csv").set_index("id").loc[game_id]
    # sample_moves = sample_submission["moves"].split(".")
    print(puzzle)
    print(f"Sample score: {len(sample_moves)}")
    
    moves = get_moves(puzzle["puzzle_type"])
    print(f"Number of moves: {len(moves)}")
    
    K = 2
    while True:
        try:
            shortest_path = get_shortest_path(moves, K, None if K == 2 else 10000000)
        except ExceedMaxSizeError:
            break
        K += 1
    print(f"K: {K}")
    print(f"Number of shortest_path: {len(shortest_path)}")
    
    current_state = puzzle["initial_state"].split(";")
    current_solution = list(sample_moves)
    initial_score = len(current_solution)
    start_time = time.time()
    
    with tqdm(total=len(current_solution) - K, desc=f"Score: {len(current_solution)} (-0)") as pbar:
        step = 0
        while step + K < len(current_solution) and time.time() - start_time < time_limit:
            replaced_moves = current_solution[step : step + K + 1]
            state_before = current_state
            state_after = current_state
            for move_name in replaced_moves:
                state_after = [state_after[i] for i in moves[move_name]]
    
            found_moves = None
            for perm, move_names in shortest_path.items():
                for i, j in enumerate(perm):
                    if state_after[i] != state_before[j]:
                        break
                else:
                    found_moves = move_names
                    break
    
            if found_moves is not None:
                length_before = len(current_solution)
                current_solution = current_solution[:step] + list(found_moves) + current_solution[step + K + 1 :]
                pbar.update(length_before - len(current_solution))
                pbar.set_description(f"Score: {len(current_solution)} ({len(current_solution) - initial_score})")
                for _ in range(K):
                    if step == 0:
                        break
                    step -= 1
                    pbar.update(-1)
                    move_name = current_solution[step]
                    move_name = move_name[1:] if move_name.startswith("-") else f"-{move_name}"
                    current_state = [current_state[i] for i in moves[move_name]]
            else:
                current_state = [current_state[i] for i in moves[current_solution[step]]]
                step += 1
                pbar.update(1)
    
    # validation
    state = puzzle["initial_state"].split(";")
    for move_name in current_solution:
        state = [state[i] for i in moves[move_name]]
    # print(puzzle["solution_state"])
    # print(state)
    assert puzzle["solution_state"].split(";") == state
    current_solution_length = len(current_solution)
    if current_solution_length < len(sample_moves):
        # print('.'.join(current_solution))
        print(f'Improvement to {game_id}')
        # Insert the moves into the database
        insert_query = "INSERT OR REPLACE INTO solutions (id, moves, count, solution_method) VALUES (?, ?, ?, ?)"
        cursor.execute(insert_query, (game_id, '.'.join(current_solution), current_solution_length, f'{solution_method}: {best_solution_method}'))
        conn.commit()
# Commit the changes and close the connection
conn.commit()
conn.close()

puzzle_type                                               globe_6/8
solution_state    A;A;D;D;G;G;J;J;M;M;P;P;S;S;V;V;A;A;D;D;G;G;J;...
initial_state     V;X;J;G;V;C;C;I;L;F;X;A;A;D;R;O;C;D;O;X;S;V;I;...
num_wildcards                                                    12
Name: 381, dtype: object
Sample score: 2783
Number of moves: 46
K: 6
Number of shortest_path: 9138736


Score: 2697 (-86): 100%|██████████| 2777/2777 [2:52:00<00:00,  3.72s/it]  


Improvement to 381
puzzle_type                                               globe_6/8
solution_state    A;A;D;D;G;G;J;J;M;M;P;P;S;S;V;V;A;A;D;D;G;G;J;...
initial_state     X;D;F;P;J;I;S;R;L;C;O;V;G;U;J;G;D;I;P;G;L;C;C;...
num_wildcards                                                     0
Name: 382, dtype: object
Sample score: 3073
Number of moves: 46
K: 6
Number of shortest_path: 9138736


Score: 3049 (-24): 100%|██████████| 3067/3067 [3:10:02<00:00,  3.72s/it]  


Improvement to 382
puzzle_type                                              globe_6/10
solution_state    A;A;A;A;D;D;D;D;G;G;G;G;J;J;J;J;M;M;M;M;A;A;A;...
initial_state     G;I;L;I;J;D;L;O;A;G;A;O;C;M;D;F;F;C;M;C;J;D;A;...
num_wildcards                                                     8
Name: 383, dtype: object
Sample score: 3293
Number of moves: 54
K: 5
Number of shortest_path: 768080


...

puzzle_type                                              globe_8/25
solution_state    A;A;A;A;A;D;D;D;D;D;G;G;G;G;G;J;J;J;J;J;M;M;M;...
initial_state     V;P;F;L;P;X;O;A;J;b;V;Y;D;Y;C;X;J;F;U;G;d;L;b;...
num_wildcards                                                     0
Name: 396, dtype: object
Sample score: 26362
Number of moves: 118
K: 4
Number of shortest_path: 269484


Score: 26314 (-48): 100%|██████████| 26358/26358 [35:21<00:00, 12.43it/s]


Improvement to 396
puzzle_type                                              globe_8/25
solution_state    A;A;A;A;A;D;D;D;D;D;G;G;G;G;G;J;J;J;J;J;M;M;M;...
initial_state     V;O;a;I;a;F;C;D;C;d;I;O;U;F;F;G;L;I;Y;Y;X;X;a;...
num_wildcards                                                     0
Name: 397, dtype: object
Sample score: 20689
Number of moves: 118
K: 4
Number of shortest_path: 269484


Score: 20663 (-26): 100%|██████████| 20685/20685 [29:15<00:00, 11.78it/s]


Improvement to 397


In [6]:
current_solution

['-r1',
 'f23',
 'f24',
 '-r2',
 'f38',
 'f19',
 'f24',
 'f19',
 'f18',
 'f29',
 'f48',
 'f2',
 'f49',
 'f0',
 'f43',
 'f7',
 'f11',
 'f5',
 'f16',
 'f4',
 'f12',
 'f48',
 'f29',
 'f18',
 'f42',
 'f5',
 'f3',
 'f49',
 'f41',
 'f43',
 'f7',
 'f11',
 'f4',
 'r8',
 'f18',
 'f32',
 'f2',
 'f48',
 'f28',
 'f4',
 'f23',
 '-r3',
 'f22',
 'f15',
 'f28',
 'f48',
 'f35',
 'f28',
 'f22',
 'f15',
 'f22',
 'r3',
 'f23',
 'f4',
 'f2',
 'f29',
 'f24',
 'f25',
 'f2',
 '-r0',
 'f48',
 'f28',
 'f7',
 'f1',
 'f11',
 'f7',
 'f43',
 '-r0',
 'f12',
 'f48',
 'f29',
 'f18',
 'f19',
 'f44',
 'r2',
 'f21',
 'r6',
 'f4',
 'f9',
 'f23',
 'f8',
 'f5',
 'f29',
 'f48',
 'f12',
 'f44',
 'f23',
 '-r3',
 'f40',
 'f25',
 'f27',
 'f14',
 'f2',
 'f28',
 'f10',
 '-r1',
 'f24',
 'f29',
 'f2',
 'f5',
 'f36',
 'f12',
 'f7',
 'f20',
 'f27',
 'f25',
 'f40',
 'r3',
 'f23',
 'f44',
 'f12',
 'f48',
 'f29',
 'f5',
 'f8',
 'f23',
 'f9',
 'f4',
 '-r6',
 'f21',
 'f2',
 'f49',
 'f48',
 'f12',
 'f23',
 'f8',
 'f5',
 'f18',
 'f32',
 'f2'

Something to consider if you try to extrapolate the relabeling process for cubes with N > 3:

For N=3, the solution state given can have only one representation as 6 sided cube which each face has the same color (excluding rotational symmetries because they are not valid solutions in this competition). It is for this reason my relabeling scheme works. 

However, consider N=4. There are 4 inner squares now, and they cannot be uniquely identified in the initial state like the N=3 case. I can confirm that the NxNxN solvers will solve cubes in terms of the faces being uniform in "color," but those states are not necessarily solutions to the puzzle. (Close but not quite.)

For N=5, the issue becomes slightly different because 5 is odd. There are now 9 inner squares6